In [1]:
# set conda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# install psi4
!conda install -q -y psi4 python=3.7 -c psi4

# install rdkit
!conda install -q -y rdkit python=3.7 -c rdkit

# set path
import sys
sys.path.append("/usr/local/lib/python3.7/site-packages/")

# this command is needed to avoid "Loader" error.
!pip install distributed

--2022-04-25 12:05:33--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2022-04-25 12:05:33--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75660608 (72M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  72.16M  93.4MB/s    in 0.8s    

2022-04-25 12:05:34 (93.4 MB/s) - ‘Miniconda3-latest-Linu

In [2]:
# import Psi4
import psi4
# check Psi4 version
print(psi4.__version__)

1.5


In [3]:
from rdkit import rdBase, Chem
from rdkit.Chem import AllChem, Draw, PandasTools
from rdkit.Chem.Draw import IPythonConsole
# check rdkit version
print('rdkit version: ', rdBase.rdkitVersion)

rdkit version:  2020.09.1


In [4]:
## convert SMILES to xyz format
def smi2xyz_neutral(smiles):
    mol = Chem.AddHs(Chem.MolFromSmiles(smiles)) # make mol object and add Hydrogen
    AllChem.EmbedMolecule(mol, AllChem.ETKDGv3()) # expand to 3D
    AllChem.MMFFOptimizeMolecule(mol) # structure optimization by MMFF
    #xyz = Chem.MolToXYZBlock(mol)
    conf = mol.GetConformer(-1)

    plus = smiles.count('+')
    minus = smiles.count('-')
    charge = plus - minus + 0 # total charge
    
    multi = (charge%2) + 1 # 2S+1 = 2*((1/2)*(charge%2)) + 1
    
    xyz = str(charge) + " " + str(multi)
    for atom, (x,y,z) in zip(mol.GetAtoms(), conf.GetPositions()):
        xyz += '\n'
        xyz += '{} {} {} {}'.format(atom.GetSymbol(), x, y, z)
        #xyz += '{}\t{}\t{}\t{}'.format(atom.GetSymbol(), x, y, z)
        
    return xyz

In [5]:
#smiles = 'COCCc1ccccc1'
smiles = 'N' # test NH3

In [6]:
xyz_neutral = smi2xyz_neutral(smiles)
mol_neutral = psi4.geometry(xyz_neutral)
print(mol_neutral.save_string_xyz())

0 1
 N    0.001022742083    0.001073332023    0.069971008798
 H    0.913793209037   -0.203010866376   -0.334455412493
 H   -0.635332609217   -0.701374299907   -0.304167325174
 H   -0.292670936557    0.889471915209   -0.333578967018



In [7]:
psi4.core.set_output_file("log_neutral.txt")

psi4.set_memory('10GB')
psi4.set_num_threads(2)

# input (PCM Solver)
pcm = '''
units = angstrom
medium
{
    solvertype = iefpcm
    solvent = acetonitrile
}
cavity
{
    type = gepol
    scaling = True
    radiiset = uff
    mode = implicit
    area = 0.3
}
'''
# acetonitrile solvet (epsilon = 37.5)

#psi4.pcm_helper(pcm)
#RHF, ROHF, UHF, CUHF, RKS, UKS (Default: RHF)
#psi4.set_options({'pcm': True, 'pcm_scf_type': 'total', 
#                  'scf_type': 'pk', 'reference': 'UHF',
#                  'e_convergence':4, 'd_convergence':4}) # HF, MP2 
#level = 'HF/STO-3G' # RHF, ROHF, UHF, CUHF
#level = 'MP2/6-31G' # RHF, ROHF, UHF, CUHF
#level = 'MP2.5/6-31G' # RHF, ROHF, UHF, CUHF
#level = 'MP3/6-31G' # RHF, ROHF, UHF, CUHF
#level = 'CCSD(T)/cc-pVTZ' # RHF, ROHF, UHF, CUHF

psi4.set_options({'pcm': True, 'pcm_scf_type': 'total', 
                  'scf_type': 'pk', 'reference': 'UKS',
                  'e_convergence':4, 'd_convergence':4}) # DFT
level = 'B3LYP/6-31+G(d,p)' # RKS, UKS

psi4.pcm_helper(pcm)

#scf_neutral_e, neutral_wfn = psi4.energy(level, return_wfn = True, molecule=mol_neutral)
scf_neutral_e, neutral_wfn = psi4.optimize(level, return_wfn = True, molecule=mol_neutral, dertype='energy')
scf_neutral_e, neutral_wfn = psi4.frequency(level, return_wfn = True, molecule=mol_neutral, ref_gradient=neutral_wfn.gradient())
print('energy at the {} level of theory:\t{:.4f} [Ha]'.format(level, scf_neutral_e))

Performing finite difference calculations
 13 displacements needed ... 1 2 3 4 5 6 7 8 9 10 11 12 13
 13 displacements needed ... 1 2 3 4 5 6 7 8 9 10 11 12 13
 13 displacements needed ... 1 2 3 4 5 6 7 8 9 10 11 12 13
Optimizer: Optimization complete!
 43 displacements needed.
 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43
energy at the B3LYP/6-31+G(d,p) level of theory:	-56.5717 [Ha]


In [8]:
!cat log_neutral.txt

ストリーミング出力は最後の 5000 行に切り捨てられました。
    Full point group: C1

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z               Mass       
    ------------   -----------------  -----------------  -----------------  -----------------
         N            0.000981954511     0.001028299495     0.067324308087    14.003074004430
         H            0.918356870311    -0.202257409727    -0.322279673438     1.007825032230
         H           -0.639377755955    -0.704956698480    -0.291767663783     1.007825032230
         H           -0.292622734280     0.892926554843    -0.321380180737     1.007825032230

  Running in c1 symmetry.

  Rotational constants: A =      9.91271  B =      9.86426  C =      6.25656 [cm^-1]
  Rotational constants: A = 297175.60362  B = 295723.20408  C = 187566.85593 [MHz]
  Nuclear repulsion =   11.894335428794204

  Charge       = 0
  Multiplicity = 1
  Electrons    = 10
  Nalpha       = 5
  

In [9]:
ld_neutral = open('log_neutral.txt')
lines = ld_neutral.readlines()
for line in lines:
    if line.find("Total") >= 0 and line.find("H,") >= 0:
        H_neutral = line.split()[6]
    if line.find("Total") >= 0 and line.find("G,") >= 0:
        G_neutral = line.split()[7]
print("Total H [Ha], Enthalpy at  298.15 [K] =",H_neutral)
print("Total G [Ha], Free enthalpy at  298.15 [K] =",G_neutral)

Total H [Ha], Enthalpy at  298.15 [K] = -56.53507499
Total G [Ha], Free enthalpy at  298.15 [K] = -56.55800271


In [10]:
xyz_oxidized = mol_neutral.save_string_xyz().replace('0 1', '1 2', 1) 
mol_oxidized = psi4.geometry(xyz_oxidized)
print(mol_oxidized.save_string_xyz())

1 2
 N    0.000981954511    0.001028299495    0.067026371638
 H    0.918357049030   -0.203795849011   -0.320863961207
 H   -0.638104646687   -0.704462236675   -0.290437378623
 H   -0.293896022266    0.893970532322   -0.319986544752



In [11]:
psi4.core.set_output_file("log_oxidized.txt")
#scf_oxidized_e, oxidized_wfn = psi4.energy(level, return_wfn = True, molecule=mol_oxidized)
#scf_oxidized_e, oxidized_wfn = psi4.optimize(level, return_wfn = True, molecule=mol_oxidized, dertype='energy')
scf_oxidized_e, oxidized_wfn = psi4.frequency(level, return_wfn = True, molecule=mol_oxidized)
print('energy at the {} level of theory:\t{:.4f} [Ha]'.format(level, scf_oxidized_e))

Performing finite difference calculations
 13 displacements needed ... 1 2 3 4 5 6 7 8 9 10 11 12 13
 43 displacements needed.
 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43
energy at the B3LYP/6-31+G(d,p) level of theory:	-56.2769 [Ha]


In [12]:
!cat log_oxidized.txt


PCM analytic gradients are not implemented yet, re-routing to finite differences.

Scratch directory: /tmp/

PCM analytic gradients are not implemented yet, re-routing to finite differences.
gradient() will perform gradient computation by finite difference of analytic energies.

         ----------------------------------------------------------
                                   FINDIF
                     R. A. King and Jonathon Misiewicz
         ---------------------------------------------------------

  Using finite-differences of energies to determine gradients.
    Generating geometries for use with 3-point formula.
    Displacement size will be 5.00e-03.
    Number of atoms is 4.
    Number of symmetric SALCs is 6.
    Translations projected? 1. Rotations projected? 1.
    Number of geometries (including reference) is 13.

  //>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>//
  //    Loading displacement 1 of 13   //
  //<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<//

Scratch directory: /tmp/
  P

In [13]:
ld_oxidized = open('log_oxidized.txt')
lines = ld_oxidized.readlines()
for line in lines:
    if line.find("Total") >= 0 and line.find("H,") >= 0:
        H_oxidized = line.split()[6]
    if line.find("Total") >= 0 and line.find("G,") >= 0:
        G_oxidized = line.split()[7]
print("Total H [Ha], Enthalpy at  298.15 [K] =",H_oxidized)
print("Total G [Ha], Free enthalpy at  298.15 [K] =",G_oxidized)

Total H [Ha], Enthalpy at  298.15 [K] = -56.22200223
Total G [Ha], Free enthalpy at  298.15 [K] = -56.24550363


In [14]:
NHE = 4.28 # [V]
# F = 96485.3321233100184 # [C/mol] = [(J/V)/mol]
# 1 [J/mol] = 1.03636E-05 [eV]
F = 96485.3321233100184*1.03636E-05 # [e]
# 1 [Ha] = 13.6058*2 [eV], Ha = Hartree
#Eox = (scf_oxidized_e - scf_neutral_e)*13.6058*2/(1*F) - NHE # The right way is to use Gibbs free energy
Eox = (float(G_oxidized) - float(G_neutral))*13.6058*2/(1*F) - NHE
print('Eox = {:.4f} [V]'.format(Eox))

Eox = 4.2241 [V]
